In [1]:
import tensorflow
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#read the dataset
import pandas as pd
train_input = pd.read_csv("data/X_train.csv")
train_output = pd.read_csv("data/y_train.csv")
test_input = pd.read_csv("data/X_test.csv")

In [3]:
num_of_train_samples = len(train_input)
num_of_features = len(train_input.loc[0]) - 1 # first one is for id
print( num_of_features)
print( num_of_train_samples)

16382
10


In [4]:
print(train_input.describe())
print(train_output.describe())

             id          x0          x1          x2          x3          x4  \
count  10.00000   10.000000   10.000000   10.000000   10.000000   10.000000   
mean    4.50000   39.600000   39.400000   38.600000   37.800000   35.500000   
std     3.02765  259.779051  263.550628  266.452914  270.179693  273.307417   
min     0.00000 -454.000000 -443.000000 -426.000000 -411.000000 -396.000000   
25%     2.25000  -61.250000  -96.000000 -131.250000 -164.250000 -184.000000   
50%     4.50000   10.500000   16.000000   21.000000   28.000000   37.500000   
75%     6.75000  220.500000  257.000000  280.250000  298.750000  282.750000   
max     9.00000  394.000000  380.000000  369.000000  358.000000  378.000000   

               x5          x6          x7          x8   ...    x16372  x16373  \
count   10.000000   10.000000   10.000000   10.000000   ...       1.0     1.0   
mean    33.800000   34.200000   35.700000   36.900000   ...     -47.0   -47.0   
std    271.379521  268.516004  265.686135  26

In [5]:
train_input

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x16372,x16373,x16374,x16375,x16376,x16377,x16378,x16379,x16380,x16381
0,0,357,356,356,356,356,355,354,354,352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,394,380,369,357,336,308,297,292,288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,22,19,15,11,9,7,7,7,7,...,-47.0,-47.0,-46.0,-46.0,-46.0,-46.0,-47.0,-48.0,-49.0,-49.0
3,3,-1,13,27,45,66,83,88,93,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-77,-123,-170,-214,-261,-289,-304,-311,-317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,-217,-225,-231,-236,-240,-242,-245,-247,-249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,138,134,131,127,123,120,118,115,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,-14,-15,-15,-15,-16,-17,-18,-18,-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,248,298,330,358,378,387,391,393,396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,-454,-443,-426,-411,-396,-374,-346,-321,-298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print(train_input['id'].dtype)
train_input = train_input.sort_values(by=['id'])
train_input = train_input.drop(columns=['id'])
train_output = train_output.sort_values(by=['id'])
train_output = train_output.drop(columns=['id'])
test_input = test_input.sort_values(by=['id'])
test_input = test_input.drop(columns=['id'])

int64


In [7]:
# Seeing whats the total number of NaNs per feature
((train_input.isna().sum() / num_of_features * 100) > 20).sum()

0

In [8]:
len(train_input.loc[0])
# train_input = train_input.loc[:9000]
# test_input = train_input.loc[:9000]

16382

In [9]:
import numpy as np
max_size = 9000
new_data = np.zeros((len(train_input), max_size))
for i in range(0, len(train_input)):
    row = train_input.loc[i]
    index = train_input.columns.get_loc(row.last_valid_index())
    dummy = np.array(row[:index+1])
    if (max_size - len(dummy)) <= 0:
        new_data[i, :] = dummy[0:max_size]
    else:
        b = dummy[0:(max_size - len(dummy))]
        goal = np.hstack((dummy, b))
        while len(goal) != max_size:
            b = dummy[0:(max_size - len(goal))]
            goal = np.hstack((goal, b))
        new_data[i, :] = goal
train_input = new_data

In [10]:
new_data = np.zeros((len(test_input), max_size))
for i in range(0, len(test_input)):
    row = test_input.loc[i]
    index = test_input.columns.get_loc(row.last_valid_index())
    dummy = np.array(row[:index+1])
    if (max_size - len(dummy)) <= 0:
        new_data[i, :] = dummy[0:max_size]
    else:
        b = dummy[0:(max_size - len(dummy))]
        goal = np.hstack((dummy, b))
        while len(goal) != max_size:
            b = dummy[0:(max_size - len(goal))]
            goal = np.hstack((goal, b))
        new_data[i, :] = goal
test_input = new_data

In [13]:
print(train_input.shape)
print(test_input.shape)

(10, 9000)
(10, 9000)


In [14]:
print(((pd.DataFrame(test_input).isna().sum() / num_of_features * 100) > 0).sum())
print(((pd.DataFrame(train_input).isna().sum() / num_of_features * 100) > 0).sum())

0
0


In [15]:
train_input = np.array(train_input)
test_input = np.array(test_input)
train_output = np.array(train_output)
check = 6000 #dont know
big = check
Label_set = np.zeros((len(train_input), 4)) #Creating of one-hot encodings representation
for i in range(len(train_output)):
    dummy = np.zeros((4))
    dummy[int(train_output[i])] = 1
    Label_set[i, :] = dummy
train_input = (train_input - train_input.mean())/(train_input.std()) #Some normalization here
train_input = np.expand_dims(train_input, axis=2) #For Keras's data input size
values = [i for i in range(len(train_output))]
permutations = np.random.permutation(values) #shuffling
train_input = train_input[permutations, :]
Label_set = Label_set[permutations, :]
Y_train = Label_set
test_input = (test_input - test_input.mean())/(test_input.std()) #Some normalization here
test_input = np.expand_dims(test_input, axis=2) #For Keras's data input size

In [26]:
print(test_input.shape)
print(train_input.shape)
print(Label_set.shape)

(10, 9000, 1)
(10, 9000, 1)
(10, 4)


In [27]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, LSTM, Conv1D, GlobalAveragePooling1D, MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import os.path
output_dir = './output'
X_train, X_val, Y_train, Y_val = train_test_split(train_input, Label_set, test_size=0.2)
def change(x):  #From boolean arrays to decimal arrays
    answer = np.zeros((np.shape(x)[0]))
    for i in range(np.shape(x)[0]):
        max_value = max(x[i, :])
        max_index = list(x[i, :]).index(max_value)
        answer[i] = max_index
    return answer.astype(np.int)

def create_model():
    model = Sequential()
    model.add(Conv1D(128, 5, activation='relu', input_shape=(9000, 1)))
    model.add(MaxPooling1D(3))
    model.add(Dropout(0.5))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Dropout(0.5))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Dropout(0.5))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Dropout(0.5))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Dropout(0.5))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(GlobalAveragePooling1D())	
    model.add(Dense(256, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    checkpointer = ModelCheckpoint(filepath=os.path.join(output_dir, 'Best_model.h5'), monitor='val_acc', verbose=1, save_best_only=True)
    hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=2, epochs=200, verbose=2, shuffle=True, callbacks=[checkpointer])
    val_loss = hist.history['loss']
    val_acc = hist.history['val_acc']
    acc = hist.history['acc']
#     pd.DataFrame(acc).to_csv(path_or_buf='Conv_models/Train_acc.csv')
#     pd.DataFrame(val_acc).to_csv(path_or_buf='Conv_models/Val_acc.csv')
#     pd.DataFrame(val_loss).to_csv(path_or_buf='Conv_models/Val_loss.csv')
#     pd.DataFrame(hist.history).to_csv(path_or_buf='Conv_models/History.csv')
    predictions = model.predict(X_val)
    score = accuracy_score(change(Y_val), change(predictions))
    print('Last epoch\'s validation score is ', score)
    df = pd.DataFrame(change(predictions))
#     df.to_csv(path_or_buf='Conv_models/Preds_' + str(format(score, '.4f')) + '.csv', index=None, header=None)
#     pd.DataFrame(confusion_matrix(change(Y_val), change(predictions))).to_csv(path_or_buf='Conv_models/Result_Conf' + str(format(score, '.4f')) + '.csv', index=None, header=None)



In [28]:
create_model()

Train on 8 samples, validate on 2 samples
Epoch 1/200
 - 2s - loss: 1.2792 - acc: 0.6250 - val_loss: 1.3186 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to ./output/Best_model.h5
Epoch 2/200
 - 0s - loss: 1.1592 - acc: 0.7500 - val_loss: 1.2842 - val_acc: 0.5000

Epoch 00002: val_acc did not improve from 0.50000
Epoch 3/200
 - 0s - loss: 0.6830 - acc: 0.6250 - val_loss: 1.3228 - val_acc: 0.5000

Epoch 00003: val_acc did not improve from 0.50000
Epoch 4/200
 - 0s - loss: 1.3215 - acc: 0.6250 - val_loss: 1.3112 - val_acc: 0.5000

Epoch 00004: val_acc did not improve from 0.50000
Epoch 5/200
 - 0s - loss: 0.7184 - acc: 0.5000 - val_loss: 1.3121 - val_acc: 0.5000

Epoch 00005: val_acc did not improve from 0.50000
Epoch 6/200
 - 0s - loss: 0.9572 - acc: 0.2500 - val_loss: 1.3293 - val_acc: 0.5000

Epoch 00006: val_acc did not improve from 0.50000
Epoch 7/200
 - 0s - loss: 0.7860 - acc: 0.7500 - val_loss: 1.3373 - val_acc: 0.5000

Epoch 00007: val_acc d

KeyboardInterrupt: 

In [ ]:
from biosppy.signals import ecg
inputs = 60 
new_data = np.zeros((len(train_input), inputs))
train_input_n = np.array(train_input)
for i in range(0, len(train_input_n)):
#     row = train_input.loc[i]
#     index = train_input.columns.get_loc(row.last_valid_index())
#     dummy = np.array(row[:index+1])
    out = ecg.christov_segmenter(signal=train_input_n[i, :], sampling_rate=300.)
    A = np.hstack((0, out[0][:len(out[0]) - 1]))
    B = out[0]
    dummy = np.lib.pad(B - A, (0, inputs - len(B)), 'constant', constant_values=(0))
    new_data[i, :] = dummy

In [ ]:
import matplotlib.pyplot as plt
train_input.loc[0][:8851].plot()
plt.show()

In [ ]:
train_input.loc[256][: 8602].plot()
plt.show()

In [ ]:
average_per_feature = train_input.mean()
train_input_fill1 = train_input.fillna(average_per_feature)
train_input_fill1.loc[256][:].plot()
plt.show()

In [ ]:
import biosppy

In [ ]:
from biosppy.signals import ecg
out = ecg.ecg(signal=train_input, sampling_rate=1000., show=True)

In [ ]:
biosppy.signals.ecg.extract_heartbeats(train_input)

# PART 2

In [ ]:
train_input_fill1 = train_input.interpolate(method='linear')
train_input_fill1.loc[8][:].plot()
plt.show()
train_input.loc[8][:].plot()
plt.show()

In [ ]:
#THE second filling NANs seems like a good one, which is linear interpolation
train_input = train_input.interpolate(method='linear', axis=1)
test_input = test_input.interpolate(method='linear', axis=1)
train_output = train_output.squeeze()

In [ ]:
((train_input.isna().sum() / num_of_features * 100) > 0).sum()

In [ ]:
original_train_output_shape = train_output.shape
original_train_input_shape = train_input.shape
original_test_input_shape = test_input.shape
print(train_output.shape)
print(train_input.shape)
print(test_input.shape)

In [ ]:
#NORMALIZING PARAMS
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_input, train_output, test_size=0.2)
print(X_train.shape)
print(X_test.shape)

In [ ]:
#SCALING
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Training Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 256

# Network Parameters
num_input = 18154 # MNIST data input (img shape: 28*28)
num_classes = 4 # MNIST total classes (0-9 digits)
dropout = 0.25 # Dropout, probability to drop a unit

In [ ]:
import tensorflow as tf

In [ ]:
tf.logging.set_verbosity(tf.logging.DEBUG)

In [ ]:
# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']

        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 18154, 1])
        conv1 = tf.layers.conv1d(x, 128, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling1d(conv1, 2, 2)
        conv1 = tf.layers.dropout(conv1, rate=dropout, training=is_training)

#         # Convolution Layer with 64 filters and a kernel size of 3
#         conv2 = tf.layers.conv1d(conv1, 128, 5, activation=tf.nn.relu)
#         # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
#         conv2 = tf.layers.max_pooling1d(conv2, 2, 2)
#         conv2 = tf.layers.dropout(conv2, rate=dropout, training=is_training)

#         # Convolution Layer with 64 filters and a kernel size of 3
#         conv3 = tf.layers.conv1d(conv2, 128, 5, activation=tf.nn.relu)
#         # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
#         conv3 = tf.layers.max_pooling1d(conv3, 2, 2)
#         conv3 = tf.layers.dropout(conv3, rate=dropout, training=is_training)
        
        
#         # Convolution Layer with 64 filters and a kernel size of 3
#         conv4 = tf.layers.conv1d(conv3, 128, 5, activation=tf.nn.relu)
#         # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
#         conv4 = tf.layers.max_pooling1d(conv4, 2, 2)
#         conv4 = tf.layers.dropout(conv4, rate=dropout, training=is_training)
        
#         # Convolution Layer with 64 filters and a kernel size of 3
#         conv4 = tf.layers.conv1d(conv4, 128, 5, activation=tf.nn.relu)
#         # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
#         conv5 = tf.layers.max_pooling1d(conv5, 2, 2)
#         conv4 = tf.layers.dropout(conv4, rate=dropout, training=is_training)
        
#         # Convolution Layer with 64 filters and a kernel size of 3
#         conv6 = tf.layers.conv1d(conv5, 128, 5, activation=tf.nn.relu)
#         # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
#         conv6 = tf.layers.max_pooling1d(conv6, 2, 2)
        
        
#         conv6 = tf.layers.average_pooling1d(conv6, 2, 2)
        
#         conv6 = tf.layers.dropout(conv6, rate=dropout, training=is_training)
#         # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv1)

        # Fully connected layer (in tf contrib folder for now)
        fc1 = tf.layers.dense(fc1, 256)
#         # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        fc2 = tf.layers.dense(fc1, 128)
        fc2 = tf.layers.dropout(fc2, rate=dropout, training=is_training)
        
        fc3 = tf.layers.dense(fc2, 64)
        fc3 = tf.layers.dropout(fc3, rate=dropout, training=is_training)

#         Output layer, class prediction
        out = tf.layers.dense(fc3, n_classes)

    return out

In [ ]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    logging_hook = tf.train.LoggingTensorHook({"loss" : loss_op, 
    "accuracy" : acc_op}, every_n_iter=1)
    print(acc_op)
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op},
      training_hooks= [logging_hook])

    return estim_specs

In [ ]:
#Build the Estimator

model = tf.estimator.Estimator(model_fn)

In [ ]:
# from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': np.array(X_train)}, y=np.array(y_train),
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

In [ ]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': np.array(X_test)}, shuffle=False)
# Use the model to predict the images class
preds = list(model.predict(input_fn))

In [ ]:
from sklearn.metrics import f1_score
preds = np.array(preds)
test_acc = f1_score(preds, np.array(y_test), average='micro')
preds
print(test_acc)

In [ ]:
#TEST ON TESTING DATASET RESERVED
#predict now
test_input = scaler.transform(test_input)
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': np.array(test_input)}, shuffle=False)
# Use the model to predict the images class
test_input_pred = list(model.predict(input_fn))
# test_input_pred = test_input_pred.squeeze()
# test_input_pred.shape

In [ ]:
#write the output
#interpolate
#Scale data
predicted_output = {'y': test_input_pred}
predicted_output_df = pd.DataFrame(data=predicted_output)
predicted_output_df.to_csv("task3/y_test.csv", index_label='id')

In [ ]:
len(test_input_pred)

In [ ]:
len(test_input_pred)

In [ ]:
train_input = train_input.interpolate()
((train_input.isna().sum() / num_of_features * 100) > 0).sum()

In [ ]:
for col in train_input:
    train_input[col] = pd.to_numeric(train_input[col], errors='coerce')

In [ ]:
train_input = train_input.interpolate(axis=1)
((train_input.isna().sum() / num_of_features * 100) > 0).sum()

In [ ]:
from sklearn.svm import SVC
model = SVC(C=0.5,kernel='rbf', gamma='scale', probability=True)
model.fit(train_input, np.array(train_output))

In [ ]:
test_input_pred = model.predict(test_input)

In [ ]:
predicted_output = {'y': test_input_pred}
predicted_output_df = pd.DataFrame(data=predicted_output)
predicted_output_df.to_csv("task3/y_test.csv", index_label='id')